# Fetch Companies listed on PSX

In [67]:
from itertools import chain

import requests
from bs4 import BeautifulSoup

## Fetch Sectors

In [4]:
URL = "https://www.psx.com.pk/psx/resources-and-tools/listings/listed-companies"
LISTING_CLASS = "notice-update-div"

In [5]:
response = requests.get(URL)
if not response.status_code == 200:
    raise ValueError(f"Error! Could not fetch page from {URL}.")

page = response.text
soup = BeautifulSoup(page, "lxml")

In [6]:
container = soup.find(class_=LISTING_CLASS)

In [7]:
xid_container = container.find("input", attrs={"id": "XID"})
if not xid_container:
    raise ValueError("Error! Could not find the XID..")

xid = xid_container.get("value")

In [24]:
sector_container = container.find("select", attrs={"name": "sector"})
if not sector_container:
    raise ValueError(f"Error! Could not find the sectors section in the site {URL}.")

sectors = sector_container.find_all("option")
if not sectors:
    raise ValueError("Error! There aren't any sectors in the sectors section.")

sector_dict = {sector.get("value"): sector.text.strip() for sector in sectors if sector.get("value") != "0"}

## Fetch Companies

In [ ]:
COMPANY_URL = "https://www.psx.com.pk/psx/custom-templates/companiesSearch-sector"

In [53]:
# xid_2 = "46afd319ff0b4dfaac5ee7de04a083f878961d1b"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": URL,
}

In [63]:
results = []

for value, sector in sector_dict.items():
    params = {"sector": value, "XID": xid}
    response = requests.get(COMPANY_URL, headers=headers, params=params)
    if not response.status_code == 200:
        raise ValueError(f"Error! Could not fetch companies against sector: {sector}")
    
    results.append(response.json())

results = list(chain.from_iterable(results))

## Fetch Addresses

In [72]:
ADDRESS_URL = "https://www.psx.com.pk/psx/resources-and-tools/Address-Book"

In [79]:
address_result = []
for company in results:
    params = {"adrress": company.get("symbol_code"), "XID": xid}
    response = requests.get(ADDRESS_URL, headers=headers, params=params)
    if not response.status_code == 200:
        raise ValueError(f"Error! Could not fetch companies against sector: {sector}")
    
    address_result.append(response.text.strip())
    break

In [80]:
address_result

['<tr><td colspan="6" class="tableHead">Company Address Book</td></tr>\n  <tr><td colspan="6" class="tableSubHead">Al-Ghazi Tractors Limited</td></tr>\n   \n    <tr> \n        <tr><td>Representative</td>\n                <td>Mr. Sakib Eltaff</td>\n                </tr>\n                <tr><td>Designation</td>\n                <td>Chief Executive Officer</td>\n                </tr>\n                <tr><td>Company</td>\n                <td>Al-Ghazi Tractors Limited</td>\n                </tr>\n                <tr><td>Address</td>\n                <td>Tractor House, Plot No.102-B, 16th East Street , Phase-1, D.H.A., Off: Korangi Road,Karachi</td>\n                </tr>\n                <tr><td>Phone</td>\n                <td>021-35318901-5</td>\n                </tr>\n                 <tr><td>Phone 2</td>\n                <td></td>\n                </tr>\n                <tr><td>Fax</td>\n                <td>021-35689387-35660882</td>\n                </tr>\n                <tr><td>Date